In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from tqdm import tqdm

df_sentences = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/BERT_Covid/Copy of covid_sentences.csv")
df_sentences = df_sentences.set_index("Unnamed: 0")

In [ ]:
df_sentences.head()

,paper_id
Unnamed: 0,
footandmouth disease fmd an acute highly contagious viral disease in susceptible clovenhoofed animals was described 100 years ago the etiologic agent fmd virus fmdv is a positivesense singlestranded rna virus that belongs to the aphthovirus genus picornaviridae family fmdv is one of the most contagious viruses in clovenhoofed animals and can cause both acute and prolonged asymptomatic but persistent infection [1] upon infection of susceptible species fmdv proliferates rapidly and causes vesicular disease in feet and mouth,db5333b01a10f165ae516d30f9d1fbf96ab4b841
the rna virus genome of fmdv displays a very high mutation rate because the virusencoded rna polymerase lacks a proofreading mechanism [2 3] the high mutation rate of fmdv coupled with its rapid proliferation and extensive population result in the rapid evolution of this virus [4] which contributes to the existence of seven main serotypes a o c asia1 south african territories sat 1 sat2 and sat3 in addition numerous variants and subtypes have been further evolved from each serotype [1] given that crossreactivity varies antigenic diversity among these serotypes have to be considered during vaccine development [5],db5333b01a10f165ae516d30f9d1fbf96ab4b841
fmdv virion has a symmetric protein shell or capsid enclosing the genomic rna genome rna contains a positive singlestrand chain approximately 83 kb long and encodes a single long open reading frame orf of about 7 kb with two alternative initiation sites the orf is flanked by a long 5untranslated region 5utr and a short 3utr and ends with a genetically encoded polya tail [6] a genomelinked viral nonstructural protein nsp 3b also known as vpg containing 2324 amino acid aa residues is covalently bound to its 5 end although this protein is rapidly released into an infected cell and is deemed to play no part in translation initiation [7] the viral orf can be translated into a polyprotein of about 250 kda which is subsequently cleaved by two virusencoded proteinases leader l pro and 3c pro to yield structural and nsps [8 9] fig 1,db5333b01a10f165ae516d30f9d1fbf96ab4b841
the fmdv genome was completely sequenced and all cleavage sites involved in the processing of polypeptides were also identified in the past two decades generally the orf region in fmdv genome is artificially divided into four functional areas due to the different functions of mature polypeptides [10] which are shown as follows fig 1 l region which is located at 5 end to the capsid component and codes for l pro p1 region encoding a precursor for capsid polypeptide which can generate four mature capsid proteins vp4 vp2 vp3 and vp1 upon cleavage by viral protease p2 region encodes three viral proteins 2a 2b and 2c in the middle region of the genome and p3 region which encodes four viral proteins 3a 3b 3c pro and 3d pol in which 3c is a viral protease and 3d an rnadependent rna polymerase [11] actually primary polyprotein is not strictly processed into four products as the functional regions by initial protease but l pro p12a 2bc and p3 by l pro 2a and 3c pro the precursors p12a 2bc and p3 are further processed into mature viral proteins and some cleavage intermediates with relative stability such as vp0 or 1ab 3abc 3bcd 3ab and 3cd by 3c pro fig 1 usually the intermediates may perform functions other than those of their individual constituents with two alternative initiation sites the orf is flanked by a long 5untranslated region 5utr and a short 3utr 3b vpg is covalently bound to its 5 end the orf region is generally divided into four functional areas l p1 p2 and p3 due to the different functions of mature polypeptides orfencoded polyprotein is processed into four products l pro p12a 2bc and p3 by l pro 2a and 3c pro the precursors p12a 2bc and p3 are further processed into mature viral proteins and some cleavage intermediates with relative stability such as vp0 or 1ab 3abc 3bcd 3ab and 3cd by 3c pro structural proteins form the biological p

In [ ]:
df_sentences = df_sentences["paper_id"].to_dict()
df_sentences_list = list(df_sentences.keys())
len(df_sentences_list)

403341

In [ ]:
list(df_sentences.keys())[:5]

['footandmouth disease fmd an acute highly contagious viral disease in susceptible clovenhoofed animals was described 100 years ago the etiologic agent fmd virus fmdv is a positivesense singlestranded rna virus that belongs to the aphthovirus genus picornaviridae family fmdv is one of the most contagious viruses in clovenhoofed animals and can cause both acute and prolonged asymptomatic but persistent infection [1]  upon infection of susceptible species fmdv proliferates rapidly and causes vesicular disease in feet and mouth',
 'the rna virus genome of fmdv displays a very high mutation rate because the virusencoded rna polymerase lacks a proofreading mechanism [2 3]  the high mutation rate of fmdv coupled with its rapid proliferation and extensive population result in the rapid evolution of this virus [4]  which contributes to the existence of seven main serotypes a o c asia1 south african territories sat 1 sat2 and sat3 in addition numerous variants and subtypes have been further evo

In [ ]:
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]

100%|██████████| 403341/403341 [00:00<00:00, 1838446.48it/s]


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/BERT_Covid/Copy of covid_sentences_Full.csv", index_col=0)
df.head()

,paper_id,title,abstract,abstract_summary
footandmouth disease fmd an acute highly contagious viral disease in susceptible clovenhoofed animals was described 100 years ago the etiologic agent fmd virus fmdv is a positivesense singlestranded rna virus that belongs to the aphthovirus genus picornaviridae family fmdv is one of the most contagious viruses in clovenhoofed animals and can cause both acute and prolonged asymptomatic but persistent infection [1] upon infection of susceptible species fmdv proliferates rapidly and causes vesicular disease in feet and mouth,db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...
the rna virus genome of fmdv displays a very high mutation rate because the virusencoded rna polymerase lacks a proofreading mechanism [2 3] the high mutation rate of fmdv coupled with its rapid proliferation and extensive population result in the rapid evolution of this virus [4] which contributes to the existence of seven main serotypes a o c asia1 south african territories sat 1 sat2 and sat3 in addition numerous variants and subtypes have been further evolved from each serotype [1] given that crossreactivity varies antigenic diversity among these serotypes have to be considered during vaccine development [5],db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...
fmdv virion has a symmetric protein shell or capsid enclosing the genomic rna genome rna contains a positive singlestrand chain approximately 83 kb long and encodes a single long open reading frame orf of about 7 kb with two alternative initiation sites the orf is flanked by a long 5untranslated region 5utr and a short 3utr and ends with a genetically encoded polya tail [6] a genomelinked viral nonstructural protein nsp 3b also known as vpg containing 2324 amino acid aa residues is covalently bound to its 5 end although this protein is rapidly released into an infected cell and is deemed to play no part in translation initiation [7] the viral orf can be translated into a polyprotein of about 250 kda which is subsequently cleaved by two virusencoded proteinases leader l pro and 3c pro to yield structural and nsps [8 9] fig 1,db5333b01a10f165ae516d30f9d1fbf96ab4b841,Biological function of Foot-and-mouth<br>dise...,footandmouth disease virus fmdv represses host...,Foot-and-mouth disease virus (FMDV)<br>repres...
the fmdv genome was completely sequenced and all cleavage sites involved in the processing of polypeptides were also identified in the past two decades generally the orf region in fmdv genome is artificially divided into four functional areas due to the different functions of mature polypeptides [10] which are shown as follows fig 1 l region which is located at 5 end to the capsid component and codes for l pro p1 region encoding a precursor for capsid polypeptide which can generate four mature capsid proteins vp4 vp2 vp3 and vp1 upon cleavage by viral protease p2 region encodes three viral proteins 2a 2b and 2c in the middle region of the genome and p3 region which encodes four viral proteins 3a 3b 3c pro and 3d pol in which 3c is a viral protease and 3d an rnadependent rna polymerase [11] actually primary polyprotein is not strictly processed into four products as the functional regions by initial protease but l pro p12a 2bc and p3 by l pro 2a and 3c pro the precursors p12a 2bc and p3 are further processed into mature viral proteins and some cleavage intermediates with relative stability such as vp0 or 1ab 3abc 3bcd 3ab and 3cd by 3c pro fig 1 usually the intermediates may perform functions other than those of their individual constituents with two alternative initiation sites the orf is flanked by a long 5untranslated region 5utr and a short 3utr 3b vpg is covalently bound to its 5 

In [ ]:
pip install -U sentence-transformers

     |████████████████████████████████| 81kB 4.9MB/s 
     |████████████████████████████████| 2.3MB 13.5MB/s 
     |████████████████████████████████| 1.2MB 35.4MB/s 
     |████████████████████████████████| 901kB 49.2MB/s 
     |████████████████████████████████| 3.3MB 40.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-cp37-none-any.whl size=123339 sha256=06cc6ce1e38662774f09c027d6b52df653f99f866f2453017eb58743d96b1875
  Stored in directory: /root/.cache/pip/wheels/0f/06/f7/faaa96fdda87462b4fd5c47b343340e9d5531ef70d0eef8242
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
import scipy.spatial
import pickle as pkl
import time

# starting time
start = time.time()
embedder = SentenceTransformer('twmkn9/albert-base-v2-squad2')
model = SentenceTransformer('twmkn9/albert-base-v2-squad2')

corpus = df_sentences_list
#corpusalbert_embeddings = embedder.encode(corpus,show_progress_bar=True)
#with open("/content/drive/MyDrive/Colab Notebooks/BERT_Covid/corpusalbert_embeddings.pkl", 'wb') as file_:
  #corpusalbert_embeddings.dump(file_)
with open("/content/drive/MyDrive/Colab Notebooks/BERT_Covid/corpusalbert_embeddings.pkl" , "rb") as file_:
  corpusalbert_embeddings = pkl.load(file_)

# Query sentences:
queries = ['Foot-and-mouth disease virus (FMDV)',
           'footandmouth disease fmd an acute highly contagious viral disease in susceptible clovenhoofed animals was described 100 years ago the etiologic agent fmd virus',
           'viral andor bacterial detection was reported as percentage positive overall and within selected categories eg age groups and seasons',
           'demographic and clinical data as well as the use of antibiotics before and during admission were collected using standard data collection forms malaria testing was implemented on site using thick blood film ',
           'the abnormal measures in mets include those in body mass index body fat waist circumference blood pressure triglycerides highdensity lipoproteins total cholesterol blood glucose and hemoglobin a1c patients with mets have an elevated risk of diabetes mellitus and cardiovascular disease '
           ]
query_embeddings = embedder.encode(queries,show_progress_bar=True)
query_embeddings = model.encode(queries)
print('Sample BERT embedding vector - note includes negative values', query_embeddings[0])

closest_n = 3
print("\nTop 10 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpusalbert_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n=========================================================")
    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df.loc[df.index== corpus[idx]].to_dict()
        print("paper_id:  " , row_dict["paper_id"][corpus[idx]] , "\n")
        print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        print("Abstract_Summary:  " , row_dict["abstract_summary"][corpus[idx]] , "\n")
        print("-------------------------------------------")
time.sleep(1)
end = time.time()

# total time taken
print(f"Runtime of the program is {end - start}")


Sample BERT embedding vector - note includes negative values [ 6.58713430e-02 -2.02142924e-01 -5.71282864e-01  9.65581954e-01
  1.06901526e+00  1.05779648e-01 -9.32473361e-01  3.38880152e-01
 -3.48100126e-01  1.55736923e+00 -1.04815610e-01  1.12571752e+00
 -1.26387751e+00  3.20691526e-01 -1.44878905e-02 -6.47834361e-01
 -7.77483821e-01 -2.22688645e-01 -2.37599254e+00 -1.32954109e+00
  1.20100951e+00  2.10395551e+00  2.97093606e+00 -3.70469880e+00
  1.63100094e-01  8.57778549e-01  7.82461651e-03  6.30382955e-01
 -1.33528244e+00  1.39396250e+00  2.90051401e-01  5.08621991e-01
 -9.43092227e-01  1.13169765e+00 -6.64565086e-01 -1.03916299e+00
 -8.11226666e-01  2.32096791e+00  1.12112844e+00 -4.51671392e-01
  1.30924392e+00  3.77113551e-01 -9.01814044e-01 -9.26946104e-01
  1.71524972e-01  1.45857736e-01 -7.33582675e-01  1.35034367e-01
 -6.02837145e-01  1.09882748e+00  6.77999020e-01 -4.74769533e-01
 -8.89665902e-01 -2.98015624e-02  6.25363469e-01  1.26828754e+00
 -1.36807501e+00  1.54164624

In [ ]:
from sentence_transformers import SentenceTransformer
import scipy.spatial
import pickle as pkl
import time

# starting time
start = time.time()
embedder = SentenceTransformer('twmkn9/albert-base-v2-squad2')
model = SentenceTransformer('twmkn9/albert-base-v2-squad2')

corpus = df_sentences_list
corpusalbert_embeddings = embedder.encode(corpus,show_progress_bar=True)
with open("/content/drive/MyDrive/Colab Notebooks/BERT_Covid/corpusalbert_embeddings.pkl", 'wb') as file_:
  corpusalbert_embeddings.dump(file_)
#with open("/content/drive/MyDrive/Colab Notebooks/BERT_Covid/corpus_embeddings.pkl" , "rb") as file_:
#corpus_embeddings = pkl.load(file_)

# Query sentences:
queries = ['Foot-and-mouth disease virus (FMDV)',
           'footandmouth disease fmd an acute highly contagious viral disease in susceptible clovenhoofed animals was described 100 years ago the etiologic agent fmd virus',
           'viral andor bacterial detection was reported as percentage positive overall and within selected categories eg age groups and seasons',
           'demographic and clinical data as well as the use of antibiotics before and during admission were collected using standard data collection forms malaria testing was implemented on site using thick blood film ',
           'the abnormal measures in mets include those in body mass index body fat waist circumference blood pressure triglycerides highdensity lipoproteins total cholesterol blood glucose and hemoglobin a1c patients with mets have an elevated risk of diabetes mellitus and cardiovascular disease '
           ]
query_embeddings = embedder.encode(queries,show_progress_bar=True)
query_embeddings = model.encode(queries)
print('Sample BERT embedding vector - note includes negative values', query_embeddings[0])

closest_n = 3
print("\nTop 10 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpusalbert_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n=========================================================")
    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df.loc[df.index== corpus[idx]].to_dict()
        print("paper_id:  " , row_dict["paper_id"][corpus[idx]] , "\n")
        print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        print("Abstract_Summary:  " , row_dict["abstract_summary"][corpus[idx]] , "\n")
        print("-------------------------------------------")
time.sleep(1)
end = time.time()

# total time taken
print(f"Runtime of the program is {end - start}")


Sample BERT embedding vector - note includes negative values [ 6.58713430e-02 -2.02142924e-01 -5.71282864e-01  9.65581954e-01
  1.06901526e+00  1.05779648e-01 -9.32473361e-01  3.38880152e-01
 -3.48100126e-01  1.55736923e+00 -1.04815610e-01  1.12571752e+00
 -1.26387751e+00  3.20691526e-01 -1.44878905e-02 -6.47834361e-01
 -7.77483821e-01 -2.22688645e-01 -2.37599254e+00 -1.32954109e+00
  1.20100951e+00  2.10395551e+00  2.97093606e+00 -3.70469880e+00
  1.63100094e-01  8.57778549e-01  7.82461651e-03  6.30382955e-01
 -1.33528244e+00  1.39396250e+00  2.90051401e-01  5.08621991e-01
 -9.43092227e-01  1.13169765e+00 -6.64565086e-01 -1.03916299e+00
 -8.11226666e-01  2.32096791e+00  1.12112844e+00 -4.51671392e-01
  1.30924392e+00  3.77113551e-01 -9.01814044e-01 -9.26946104e-01
  1.71524972e-01  1.45857736e-01 -7.33582675e-01  1.35034367e-01
 -6.02837145e-01  1.09882748e+00  6.77999020e-01 -4.74769533e-01
 -8.89665902e-01 -2.98015624e-02  6.25363469e-01  1.26828754e+00
 -1.36807501e+00  1.54164624